In [1]:
import random
import numpy as np
import pandas as pd
from itertools import combinations

In [2]:
def get_int_value(card) -> int:
    """
    Returns the integer value of a card
    """
    if card == 'A':
        return 14
    elif card == 'K':
        return 13
    elif card == 'Q':
        return 12
    elif card == 'J':
        return 11
    elif card == 'T':
        return 10
    else:
        return int(card)

def get_int_value_as_str(card) -> str:
    """
    Returns the integer value of a card
    """
    if card == 'A':
        return '14'
    elif card == 'K':
        return '13'
    elif card == 'Q':
        return '12'
    elif card == 'J':
        return '11'
    elif card == 'T':
        return '10'
    else:
        return card

def get_str_value(card) -> str:
    """
    Returns the string value of a card
    """
    if card == 14:
        return 'A'
    elif card == 13:
        return 'K'
    elif card == 12:
        return 'Q'
    elif card == 11:
        return 'J'
    elif card == 10:
        return 'T'
    else:
        return str(card)

In [3]:
def check_hand(hand, board):
    """Returns a int with the value of hand, the value of the highest card in the hand and the value of the highest card in the board"""

    total = hand + board

    hand = pd.DataFrame([(x[0],x[1]) for x in hand], columns=['value', 'suit'])
    hand['int_value'] = hand.value.apply(get_int_value)
    
    total = pd.DataFrame([(x[0],x[1]) for x in total], columns=['value', 'suit'])
    total['int_value'] = total.value.apply(get_int_value)

    if is_straight_flush(total):
        return [8, total.int_value.max(), hand.int_value.max()]
    elif is_four_of_a_kind(total):
        return [7, total.int_value.value_counts().idxmax(), hand.int_value.max()]
    elif is_full_house(total):
        return [6, total.int_value.value_counts().idxmax(), hand.int_value.max()]
    elif is_flush(total):
        return [5, hand.int_value.max(), hand.int_value.max()]
    elif is_straight(total)[0]:
        return [4, total.int_value.max(), hand.int_value.max()]
    elif is_three_of_a_kind(total):
        return [3, total.int_value.value_counts().idxmax(), hand.int_value.max()]
    elif is_two_pair(total):
        return [2, total.int_value.value_counts()[total.int_value.value_counts() == 2].index.max(), hand.int_value.max()]
    elif is_one_pair(total):
        return [1, total.int_value.value_counts().idxmax(), hand.int_value.max()]
    else:
        return [0, total.int_value.max(), hand.int_value.max()]
    
# check if it is a straight flush
def is_straight_flush(hand):
    """Returns a boolean if it is a straight flush"""
    straight, hand = is_straight(hand)
    if straight and is_flush(hand):
        return True
    else:
        return False

# check if it is a four of a kind
def is_four_of_a_kind(hand) -> bool:
    """Returns a boolean if it is a four of a kind"""
    if hand.value.value_counts().max() == 4:
        return True
    else:
        return False
    
# check if it is a full house
def is_full_house(hand) -> bool:
    """Returns a boolean if it is a full house"""
    if 3 in hand.value.value_counts().values and 2 in hand.value.value_counts().values:
        return True
    else:
        return False

# check if it is a flush
def is_flush(hand) -> bool:
    """Returns a boolean if it is a flush"""
    if hand.suit.value_counts().values.max() >= 5:
        return True
    else:
        return False

# check if it is a straight
def is_straight(hand):
    """Returns a boolean if it is a straight"""
    # There are 7 cards in total to check
    # There can be a straight with 5 cards

    # Check if there is a straight with 5 cards
    perm = combinations(list(hand.value + hand.suit), 5)
    for i in list(perm):
        it_is, helper = is_straight_helper(i)
        if it_is:
            return True, helper
    return False, None

def is_straight_helper(hand):
    """ Returns a boolean if it is a straight with 5 cards"""
    hand = pd.DataFrame([(x[0],x[1]) for x in hand], columns=['value', 'suit'])
    hand['int_value'] = hand.value.apply(get_int_value)
    
    if hand.int_value.max() - hand.int_value.min() == 4 and len(set(hand.int_value)) == 5:
        return True, hand
    else:
        return False, None
    

# check if it is a three of a kind
def is_three_of_a_kind(hand) -> bool:
    """Returns a boolean if it is a three of a kind"""
    if 3 in hand.value.value_counts().values:
        return True
    else:
        return False

# check if it is a two pair
def is_two_pair(hand) -> bool:
    """Returns a boolean if it is a two pair"""
    if len(hand.value.value_counts()[hand.value.value_counts() == 2]) == 2 or len(hand.value.value_counts()[hand.value.value_counts() == 2]) == 3:
        return True
    else:
        return False

# check if it is a one pair
def is_one_pair(hand) -> bool:
    """Returns a boolean if it is a one pair"""
    if 2 in hand.value.value_counts().values:
        return True
    else:
        return False

# check if it is a high card
def is_high_card(hand) -> bool:
    """Returns a boolean if it is a high card"""
    if len(set(hand.value.value_counts().values)) == 7:
        return True
    else:
        return False

# check if it works
print(check_hand(['A♠', 'K♠'],['Q♠', 'J♠', 'T♠', 'J♦', '8♥'])) # straight flush
print(check_hand(['2♠', '2♦'],[ '2♥', '2♣', 'K♠', 'Q♠', 'J♠'])) # four of a kind
print(check_hand(['A♠', 'A♦'],[ 'A♥', 'K♠', 'K♦', '8♥', 'J♠'])) # full house
print(check_hand(['9♠', 'K♠'],[ 'Q♠', 'T♠', '2♠', 'J♦', '8♠'])) # flush
print(check_hand(['A♥', 'K♦'],[ 'Q♠', 'J♥', 'T♠', 'J♦', '8♥'])) # straight
print(check_hand(['A♠', 'A♦'],[ 'A♥', 'K♠', 'Q♠', 'J♦', '8♥'])) # three of a kind
print(check_hand(['A♠', 'A♦'],[ 'K♥', 'K♠', 'Q♠', 'J♦', '8♥'])) # two pair
print(check_hand(['A♠', 'A♦'],[ 'K♥', 'Q♠', 'J♦', '8♥', '7♠'])) # one pair
print(check_hand(['A♠', '2♦'],[ 'Q♥', 'J♠', 'T♦', '3♥', '7♠'])) # high card


[8, 14, 14]
[7, 2, 2]
[6, 14, 14]
[5, 13, 13]
[4, 14, 14]
[3, 14, 14]
[2, 14, 14]
[1, 14, 14]
[0, 14, 14]


In [4]:
def create_deck():
    """Create a deck of 52 cards"""
    return [x + y for x in ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2'] for y in ['♠', '♥', '♦', '♣']]

def shuffle_deck(cards):
    """Shuffle a deck of cards"""
    random.shuffle(cards)
    return cards

def deal_cards(cards, num_players):
    """Deal cards to all players"""
    hands = [[] for _ in range(num_players)]
    _ = [hands[j].append(cards.pop(0)) for i in range(2) for j in range(num_players)] 
    return hands, cards

def get_all_possibilities(cards, num_cards):
    """Get all possible combinations of cards"""
    if num_cards not in [1, 2, 3, 4, 5]:
        raise ValueError("Invalid number of cards")
    if num_cards == 1:
        return cards
    else:
        return list(combinations(cards, num_cards))       

def get_win_probabilities(hands, board):
    """Get the probability of winning for each player"""
    cards = create_deck()
    # Remove cards that are already dealt
    _ = [cards.remove(card) for hand in hands for card in hand]
    _ = [cards.remove(card) for card in board]
    
    # Simulate all possible outcomes
    total_length = len(hands[0]) + len(board)
    possibilities = get_all_possibilities(cards, 7 - total_length)

    # Get the probability of winning for each player
    results = []
    i = 0
    for possibility in possibilities:
        print(round(i / len(possibilities) * 100, 2), '%', end='\r')
        # Add the board cards to the possibility
        possibility = list(possibility) + board
        # Get the best hand for each player
        best_hands = [check_hand(list(possibility) + hand) for hand in hands]
        # Get the winner
        winner = best_hands.index(max(best_hands))
        results.append(winner)
        i += 1
    
    # Get the probability of winning for each player
    #results = pd.DataFrame(results, columns=['hand', 'high_value'])
    return results


In [5]:
def calculate_percentages_2players(hands, boards):
    """Calculate the percentage of wins for each player"""
    test = [[''.join(hands[0])+''.join(board), ''.join(hands[1])+''.join(board)] for board in boards]
    df = pd.DataFrame(test, columns=['hand1', 'hand2'])
    df['numbers1'] = df.hand1.apply(lambda x: x[::2])
    df['numbers2'] = df.hand2.apply(lambda x: x[::2])
    df['suits1'] = df.hand1.apply(lambda x: x[1::2])
    df['suits2'] = df.hand2.apply(lambda x: x[1::2])

    # get max card of each hand
    max_hand = (max(get_int_value(hands[0][0][0]), get_int_value(hands[0][1][0])), max(get_int_value(hands[1][0][0]), get_int_value(hands[1][1][0])))
    min_hand = (min(get_int_value(hands[0][0][0]), get_int_value(hands[0][1][0])), min(get_int_value(hands[1][0][0]), get_int_value(hands[1][1][0])))
    if max_hand[0] == max_hand[1]:
        df['highest_hand'] = min_hand.index(max(min_hand)) + 1
    else:
        df['highest_hand'] = max_hand.index(max(max_hand)) + 1
        

    # Clear
    df['is_quads1'] = df.numbers1.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 4)
    df['is_quads2'] = df.numbers2.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 4)
    df['is_quads1'] = np.where(df.is_quads1, 7, 0)
    df['is_quads2'] = np.where(df.is_quads2, 7, 0)

    df_total = df[(df.is_quads1 == 7) | (df.is_quads2 == 7)]
    df = df[(df.is_quads1 == 0) & (df.is_quads2 == 0)]

    # Falta tener en cuenta el count de la mesa
    hand1_pair = hands[0][0][0] == hands[0][1][0]
    hand2_pair = hands[1][0][0] == hands[1][1][0]
    df['is_full1'] = df.numbers1.apply(lambda x: (max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 2) or
                                                (max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 3 and not hand1_pair))
    df['is_full2'] = df.numbers2.apply(lambda x: (max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 2) or 
                                                (max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 3 and not hand2_pair))
    df['is_full1'] = np.where(df.is_full1, 6, 0)
    df['is_full2'] = np.where(df.is_full2, 6, 0)

    df_total = pd.concat([df_total, df[(df.is_full1 == 6) | (df.is_full2 == 6)]])
    df = df[(df.is_full1 == 0) & (df.is_full2 == 0)]

    # Clear
    df['is_flush1'] = df.suits1.apply(lambda x: max(x.count(x[0]), x.count(x[1])) >= 5)
    df['is_flush2'] = df.suits2.apply(lambda x: max(x.count(x[0]), x.count(x[1])) >= 5)
    df['is_flush1'] = np.where(df.is_flush1, 5, 0)
    df['is_flush2'] = np.where(df.is_flush2, 5, 0)

    df_total = pd.concat([df_total, df[(df.is_flush1 == 5) | (df.is_flush2 == 5)]])
    df = df[(df.is_flush1 == 0) & (df.is_flush2 == 0)]

    # Clear
    def is_straight(x):
        x = ''.join(map(str, sorted(map(get_int_value, list(x)))))
        for h in ['23456','34567','45678','56789','678910','7891011','910111213','1011121314']:
            if h in x:
                return 4
        return 0

    df['is_straight1'] = df.numbers1.apply(is_straight)
    df['is_straight2'] = df.numbers2.apply(is_straight)

    df_total = pd.concat([df_total, df[(df.is_straight1 == 4) | (df.is_straight2 == 4)]])
    df = df[(df.is_straight1 == 0) & (df.is_straight2 == 0)]

    # Not Clear
    df['is_trips1'] = df.numbers1.apply(lambda x: (max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 1) or
                                                (max(x.count(x[0]), x.count(x[1])) == 3 and hand1_pair))
    df['is_trips2'] = df.numbers2.apply(lambda x: (max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 1) or
                                                (max(x.count(x[0]), x.count(x[1])) == 3 and hand2_pair))
    df['is_trips1'] = np.where(df.is_trips1, 3, 0)
    df['is_trips2'] = np.where(df.is_trips2, 3, 0)

    df_total = pd.concat([df_total, df[(df.is_trips1 == 3) | (df.is_trips2 == 3)]])
    df = df[(df.is_trips1 == 0) & (df.is_trips2 == 0)]

    # Clear
    df['is_two_pair1'] = df.numbers1.apply(lambda x: (max(x.count(x[0]), x.count(x[1])) == 2 and hand1_pair) or 
                                                (max(x.count(x[0]), x.count(x[1])) == 2 and min(x.count(x[0]), x.count(x[1])) == 2 and not hand1_pair))
    df['is_two_pair2'] = df.numbers2.apply(lambda x: (max(x.count(x[0]), x.count(x[1])) == 2 and hand2_pair) or
                                                (max(x.count(x[0]), x.count(x[1])) == 2 and min(x.count(x[0]), x.count(x[1])) == 2 and not hand2_pair))
    df['is_two_pair1'] = np.where(df.is_two_pair1, 2, 0)
    df['is_two_pair2'] = np.where(df.is_two_pair2, 2, 0)

    df_total = pd.concat([df_total, df[(df.is_two_pair1 == 2) | (df.is_two_pair2 == 2)]])
    df = df[(df.is_two_pair1 == 0) & (df.is_two_pair2 == 0)]

    # Clear
    df['is_pair1'] = df.numbers1.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 2 and min(x.count(x[0]), x.count(x[1])) == 1)
    df['is_pair2'] = df.numbers2.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 2 and min(x.count(x[0]), x.count(x[1])) == 1)
    df['is_pair1'] = np.where(df.is_pair1, 1, 0)
    df['is_pair2'] = np.where(df.is_pair2, 1, 0)

    df_total = pd.concat([df_total, df[(df.is_pair1 == 1) | (df.is_pair2 == 1)]])
    df = df[(df.is_pair1 == 0) & (df.is_pair2 == 0)]

    # Clear
    df['is_high1'] = 0
    df['is_high2'] = 0

    df_total = pd.concat([df_total, df[(df.is_high1 == 0) | (df.is_high2 == 0)]]).reset_index(drop=True)


    # Clear
    df_total['max_value1'] = df_total[[x for x in df_total.columns if 'is_' in x and '1' in x]].max(axis=1)
    df_total['max_value2'] = df_total[[x for x in df_total.columns if 'is_' in x and '2' in x]].max(axis=1)

    df_total['winner'] = df_total['highest_hand']
    df_total['winner'] = np.where(df_total['max_value1'] > df_total['max_value2'], 1, df_total['winner'])
    df_total['winner'] = np.where(df_total['max_value1'] < df_total['max_value2'], 2, df_total['winner'])


    results = df_total['winner'].value_counts().sort_index() / df_total.shape[0] * 100

    if results.shape[0] > 1:
        print('Player 1:', round(results.loc[1], 2))
        print('Player 2:', round(results.loc[2], 2))
    else:
        print('The winner is player number:', results.index[0])

    return df_total

In [32]:
def is_straight(x):
    x = ''.join(map(str, sorted(map(get_int_value, list(x)))))
    for h in ['23456','34567','45678','56789','678910','7891011','910111213','1011121314']:
        if h in x:
            return 4
    return 0

def calculate_percentages(hands, boards):
    """Calculate the percentage of wins for each player"""
    
    test = [[''.join(hand)+''.join(board) for hand in hands] for board in boards]
    df = pd.DataFrame(test, columns=['hand'+str(i+1) for i in range(len(hands))])

    max_hand = [max(get_int_value(hands[i][0][0]), get_int_value(hands[i][1][0])) for i in range(len(hands))]
    min_hand = [min(get_int_value(hands[i][0][0]), get_int_value(hands[i][1][0])) for i in range(len(hands))]

    if max_hand.count(max(max_hand)) == 1:
        df['highest_hand'] = max_hand.index(max(max_hand)) + 1
    else:
        indexes = [i for i, x in enumerate(max_hand) if x == max(max_hand)]
        print(min_hand.index(max([min_hand[i] for i in indexes])) + 1)
        df['highest_hand'] = min_hand.index(max([min_hand[i] for i in indexes])) + 1
        # AQUI AUN SE PUEDE AÑADIR CUANDO SEAN VALORES IGUALES

    for i in range(1, len(hands)+1):
        df['numbers'+str(i)] = df['hand'+str(i)].apply(lambda x: x[::2])
        df['suits'+str(i)] = df['hand'+str(i)].apply(lambda x: x[1::2])

        #FALTA EL STRAIGHT FLUSH

        #QUADS
        df['is_quads'+str(i)] = df['numbers'+str(i)].apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 4)
        df['is_quads'+str(i)] = np.where(df['is_quads'+str(i)], 7, 0)

        #FULL (FALTA TENER EN CUENTA EL COUNT DE LA MESA POR SI HAY PAREJAS DENTRO)
        hand_is_pair = hands[i-1][0][0] == hands[i-1][1][0]
        df['is_full'+str(i)] = df['numbers'+str(i)].apply(lambda x: (max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 2) or
                                                    (max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 3 and not hand_is_pair))
        df['is_full'+str(i)] = np.where(df['is_full'+str(i)], 6, 0)

        #FLUSH
        df['is_flush'+str(i)] = df['suits'+str(i)].apply(lambda x: max(x.count(x[0]), x.count(x[1])) >= 5)
        df['is_flush'+str(i)] = np.where(df['is_flush'+str(i)], 5, 0)

        #STRAIGHT
        df['is_straight'+str(i)] = df['numbers'+str(i)].apply(is_straight)

        #THREE OF A KIND (CREO QUE FALTA ALGÚN DETALLE)
        df['is_trips'+str(i)] = df['numbers'+str(i)].apply(lambda x: (max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 1) or
                                                    (max(x.count(x[0]), x.count(x[1])) == 3 and hand_is_pair))
        df['is_trips'+str(i)] = np.where(df['is_trips'+str(i)], 3, 0)
        
        #TWO PAIR
        df['is_two_pair'+str(i)] = df['numbers'+str(i)].apply(lambda x: (max(x.count(x[0]), x.count(x[1])) == 2 and hand_is_pair) or
                                                    (max(x.count(x[0]), x.count(x[1])) == 2 and min(x.count(x[0]), x.count(x[1])) == 2 and not hand_is_pair))
        df['is_two_pair'+str(i)] = np.where(df['is_two_pair'+str(i)], 2, 0)

        #PAIR
        df['is_pair'+str(i)] = df['numbers'+str(i)].apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 2 and min(x.count(x[0]), x.count(x[1])) == 1)
        df['is_pair'+str(i)] = np.where(df['is_pair'+str(i)], 1, 0)

        #HIGH CARD
        df['is_high'+str(i)] = 0

        #MAX VALUE
        df['max_value'+str(i)] = df[[x for x in df.columns if 'is_' in x and str(i) in x]].max(axis=1)

    #GET WINNER
    df['winner'] = df['highest_hand']
    df['winner'] = np.where(df[[x for x in df.columns if 'max' in x]].apply(lambda x: list(x).count(x.max()), axis=1) == 1,
                            df[[x for x in df.columns if 'max' in x]].idxmax(axis=1).astype(str).str[-1].astype(int),
                            df['winner']
                            )
    df['winner'] = np.where(df[[x for x in df.columns if 'max' in x]].apply(lambda x: list(x).count(x.max()), axis=1) > 1,
                            [np.where(altas == altas.max(), bajas, 0).argmax() + 1 for altas, bajas in zip(df[[x for x in df.columns if 'max' in x]].values, [max_hand]*df.shape[0])],
                            df['winner']
                            )
    df['winner'] = np.where(df['max_value1'] == df['max_value2'], df['highest_hand'], df['winner'])
    results = df['winner'].value_counts().sort_index() / df.shape[0] * 100

    if results.shape[0] > 1:
        for i in results.index:
            print('Player', i, ':', round(results.loc[i], 2))
    else:
        print('The winner is player number:', results.index[0])

    return df

In [7]:
cards = create_deck()
cards = shuffle_deck(cards)
num_players = 2
hands, cards = deal_cards(cards, num_players)

for player in range(num_players):
    print('Player', player + 1, ':', ''.join(hands[player]))

boards = get_all_possibilities(cards, 5)
df_preflop = calculate_percentages(hands, boards)

flop = [cards.pop(0) for i in range(3)]

print('Flop: ', ''.join(flop))

boards = get_all_possibilities(cards, 2)
df_flop = calculate_percentages(hands, [flop + list(board) for board in boards])

turn = [cards.pop(0)]

print('Turn: ', ''.join(turn))
boards = get_all_possibilities(cards, 1)
df_turn = calculate_percentages(hands, [flop + turn + list(board) for board in boards])

river = [cards.pop(0)]
print('River: ', ''.join(river))
df_river = calculate_percentages(hands, [flop + turn + river])

Player 1 : 2♥Q♣
Player 2 : 7♠K♦


KeyboardInterrupt: 

7 -> quads //
6 -> full house //
5 -> flush //
4 -> straight //
3 -> three of a kind //
2 -> two pair // 
1 -> one pair //
0 -> high card //
```


In [33]:
while True:
    cards = create_deck()
    cards = shuffle_deck(cards)
    num_players = 2
    hands, cards = deal_cards(cards, num_players)

    max_hand = [max(get_int_value(hands[i][0][0]), get_int_value(hands[i][1][0])) for i in range(len(hands))]

    if max_hand.count(max(max_hand)) > 1:
        break
        

for player in range(num_players):
    print('Player', player + 1, ':', ''.join(hands[player]))

boards = get_all_possibilities(cards, 5)
df_preflop = calculate_percentages(hands, boards)
print('-----OLD VERSION -----')
df_preflop2 = calculate_percentages_2players(hands, boards)

flop = [cards.pop(0) for i in range(3)]

print('Flop: ', ''.join(flop))

boards = get_all_possibilities(cards, 2)
df_flop = calculate_percentages(hands, [flop + list(board) for board in boards])
print('-----OLD VERSION -----')
df_flop2 = calculate_percentages_2players(hands, [flop + list(board) for board in boards])

turn = [cards.pop(0)]

print('Turn: ', ''.join(turn))
boards = get_all_possibilities(cards, 1)
df_turn = calculate_percentages(hands, [flop + turn + list(board) for board in boards])
print('-----OLD VERSION -----')

df_turn2 = calculate_percentages_2players(hands, [flop + turn + list(board) for board in boards])

river = [cards.pop(0)]
print('River: ', ''.join(river))
df_river = calculate_percentages(hands, [flop + turn + river])
print('-----OLD VERSION -----')
df_river2 = calculate_percentages_2players(hands, [flop + turn + river])

Player 1 : 7♦Q♥
Player 2 : 2♥Q♣
1
Player 1 : 77.62
Player 2 : 22.38
-----OLD VERSION -----
Player 1: 77.62
Player 2: 22.38
Flop:  Q♠3♠A♥
1
Player 1 : 87.88
Player 2 : 12.12
-----OLD VERSION -----
Player 1: 87.88
Player 2: 12.12
Turn:  J♥
1
Player 1 : 93.18
Player 2 : 6.82
-----OLD VERSION -----
Player 1: 93.18
Player 2: 6.82
River:  A♦
1
The winner is player number: 1
-----OLD VERSION -----
The winner is player number: 1


In [31]:
df_preflop[['hand1', 'hand2', 'max_value1', 'max_value2', 'winner']]

,hand1,hand2,max_value1,max_value2,winner
0,J♠5♥5♦8♦2♦K♥8♣,8♠J♦5♦8♦2♦K♥8♣,1,3,2
1,J♠5♥5♦8♦2♦K♥Q♦,8♠J♦5♦8♦2♦K♥Q♦,1,5,2
2,J♠5♥5♦8♦2♦K♥9♠,8♠J♦5♦8♦2♦K♥9♠,1,1,1
3,J♠5♥5♦8♦2♦K♥K♣,8♠J♦5♦8♦2♦K♥K♣,1,1,1
4,J♠5♥5♦8♦2♦K♥7♣,8♠J♦5♦8♦2♦K♥7♣,1,1,1
...,...,...,...,...,...
1712299,J♠5♥A♠4♥A♣4♣6♥,8♠J♦A♠4♥A♣4♣6♥,0,0,1
1712300,J♠5♥A♠4♥A♣5♠6♥,8♠J♦A♠4♥A♣5♠6♥,1,0,1
1712301,J♠5♥A♠4♥4♣5♠6♥,8♠J♦A♠4♥4♣5♠6♥,1,0,1
1712302,J♠5♥A♠A♣4♣5♠6♥,8♠J♦A♠A♣4♣5♠6♥,1,0,1


In [26]:
hands = [['Q♣', 'T♥'], ['2♣', 'Q♥']]

max_hand = [max(get_int_value(hands[i][0][0]), get_int_value(hands[i][1][0])) for i in range(len(hands))]
min_hand = [min(get_int_value(hands[i][0][0]), get_int_value(hands[i][1][0])) for i in range(len(hands))]

if max_hand.count(max(max_hand)) == 1:
        print(max_hand.index(max(max_hand)) + 1)
else:
    indexes = [i for i, x in enumerate(max_hand) if x == max(max_hand)]
    print(min_hand.index(max([min_hand[i] for i in indexes])) + 1)

1


In [9]:
max_hand

[12, 12]

In [10]:
min_hand

[3, 8]

In [12]:
indexes

[0, 1]

7 -> quads //
6 -> full house //
5 -> flush //
4 -> straight //
3 -> three of a kind //
2 -> two pair // 
1 -> one pair //
0 -> high card //
```

In [51]:
df[['hand1', 'hand2', 'winner', 'max_value1', 'max_value2', 'highest_hand']].loc[np.random.randint(0, df.shape[0], 10)]

,hand1,hand2,winner,max_value1,max_value2,highest_hand
150,K♠4♠3♦2♣K♦6♦T♥,Q♣Q♥3♦2♣K♦6♦T♥,2,1,2,1
93,K♠4♠3♦2♣K♦4♣9♦,Q♣Q♥3♦2♣K♦4♣9♦,1,2,2,1
682,K♠4♠3♦2♣K♦8♦2♦,Q♣Q♥3♦2♣K♦8♦2♦,2,1,2,1
523,K♠4♠3♦2♣K♦J♦Q♦,Q♣Q♥3♦2♣K♦J♦Q♦,2,1,3,1
17,K♠4♠3♦2♣K♦Q♠A♥,Q♣Q♥3♦2♣K♦Q♠A♥,2,1,3,1
637,K♠4♠3♦2♣K♦4♥Q♦,Q♣Q♥3♦2♣K♦4♥Q♦,2,2,3,1
982,K♠4♠3♦2♣K♦2♠Q♦,Q♣Q♥3♦2♣K♦2♠Q♦,2,1,3,1
962,K♠4♠3♦2♣K♦2♦9♥,Q♣Q♥3♦2♣K♦2♦9♥,2,1,2,1
56,K♠4♠3♦2♣K♦8♥7♣,Q♣Q♥3♦2♣K♦8♥7♣,2,1,2,1
503,K♠4♠3♦2♣K♦J♦T♠,Q♣Q♥3♦2♣K♦J♦T♠,2,1,2,1


In [17]:
df[['hand1', 'hand2', 'winner', 'max_value1', 'max_value2', 'highest_hand']].loc[np.random.randint(0, df.shape[0], 10)]

,hand1,hand2,winner,max_value1,max_value2,highest_hand
976,9♥2♠K♦8♠7♠J♥J♦,3♣4♠K♦8♠7♠J♥J♦,1,0,0,1
859,9♥2♠K♦8♠7♠2♦T♦,3♣4♠K♦8♠7♠2♦T♦,1,1,0,1
551,9♥2♠K♦8♠7♠5♣J♦,3♣4♠K♦8♠7♠5♣J♦,1,0,0,1
850,9♥2♠K♦8♠7♠K♥J♦,3♣4♠K♦8♠7♠K♥J♦,1,0,0,1
744,9♥2♠K♦8♠7♠6♣3♠,3♣4♠K♦8♠7♠6♣3♠,2,0,1,1
840,9♥2♠K♦8♠7♠K♥8♣,3♣4♠K♦8♠7♠K♥8♣,1,0,0,1
678,9♥2♠K♦8♠7♠4♦2♥,3♣4♠K♦8♠7♠4♦2♥,1,1,1,1
971,9♥2♠K♦8♠7♠A♣J♦,3♣4♠K♦8♠7♠A♣J♦,1,4,0,1
563,9♥2♠K♦8♠7♠6♠6♥,3♣4♠K♦8♠7♠6♠6♥,1,0,0,1
342,9♥2♠K♦8♠7♠3♥K♥,3♣4♠K♦8♠7♠3♥K♥,2,0,1,1
